### Example of a crude data-driven model based on LSTMs

In [54]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import sqlite3
from datetime import datetime, timedelta
import os

In [55]:
class WeatherDataset(Dataset):
    def __init__(self, features, targets, timestamps, sequence_length):
        self.features = torch.FloatTensor(features)
        self.targets = torch.FloatTensor(targets)
        self.timestamps = timestamps
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.features) - self.sequence_length

    def __getitem__(self, idx):
        return (
            self.features[idx:idx+self.sequence_length],
            self.targets[idx+self.sequence_length],
            self.timestamps[idx+self.sequence_length]
        )


#class WeatherDataset(Dataset):
#    def __init__(self, features, targets, sequence_length):
#        self.features = features
#       self.targets = targets
#        self.sequence_length = sequence_length#
#
#    def __len__(self):
#        return len(self.features) - self.sequence_length#
#
#    def __getitem__(self, idx):
#        return (
#            self.features[idx:idx+self.sequence_length],
#            self.targets[idx+self.sequence_length]
#        )
#Original version had an issue with the float32 data length

class WeatherDataset(Dataset):
    def __init__(self, features, targets, sequence_length):
        self.features = torch.FloatTensor(features)  # Convert to torch.float32
        self.targets = torch.FloatTensor(targets)    # Convert to torch.float32
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.features) - self.sequence_length

    def __getitem__(self, idx):
        return (
            self.features[idx:idx+self.sequence_length],
            self.targets[idx+self.sequence_length]
        )


In [56]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [57]:
def load_observation_data(db_path,variables, year):
    dfs = []
    for var in variables:
        dbase = os.path.join(db_path,f"OBSTABLE_{var}_{year}.sqlite")
        conn = sqlite3.connect(dbase)
        query = f"SELECT * FROM SYNOP"
        df = pd.read_sql_query(query, conn)
        df['variable'] = var
        dfs.append(df)
        conn.close()

    obs_data = pd.concat(dfs, axis=0)
    obs_data['valid_dttm'] = pd.to_datetime(obs_data['valid_dttm'], unit='s')
    return obs_data


In [58]:
def load_forecast_data(db_path, variables, year, month, init_time):
    dfs = []
    for var in variables:
        dbase = os.path.join(db_path,f"{year}/{month:02d}/FCTABLE_{var}_{year}{month:02d}_{init_time:02d}.sqlite")
        conn = sqlite3.connect(dbase)
        query = f"SELECT * FROM FC"
        df = pd.read_sql_query(query, conn)
        dfs.append(df)
        conn.close()

    fc_data = pd.concat(dfs, axis=0)
    fc_data['valid_dttm'] = pd.to_datetime(fc_data['valid_dttm'], unit='s')
    fc_data['fcst_dttm'] = pd.to_datetime(fc_data['fcst_dttm'], unit='s')
    return fc_data


In [59]:
def preprocess_data(obs_data, fc_data):
    # Rename columns to avoid conflicts
    obs_data = obs_data.rename(columns={'lat': 'lat_obs', 'lon': 'lon_obs', 'elev': 'elev_obs'})
    fc_data = fc_data.rename(columns={'lat': 'lat_fc', 'lon': 'lon_fc', 'model_elevation': 'elev_fc'})

    # Merge observation and forecast data
    merged_data = pd.merge(obs_data, fc_data, on=['SID', 'valid_dttm'], suffixes=('_obs', '_fc'))

    # Sort data by station and time
    merged_data = merged_data.sort_values(['SID', 'valid_dttm'])

    # Create features and target
    feature_columns = ['TROAD', 'glatmodel_det', 'elev_obs', 'elev_fc', 'lat_obs', 'lon_obs', 'lead_time']
    features = merged_data[feature_columns].values
    target = merged_data['TROAD'].values
    timestamps = merged_data['valid_dttm'].values

    # Normalize the data
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    return scaled_features, target, timestamps, scaler


#def preprocess_data(obs_data, fc_data):
#    # Merge observation and forecast data
#    merged_data = pd.merge(obs_data, fc_data, on=['SID', 'valid_dttm'], suffixes=('_obs', '_fc'))###

#    # Sort data by station and time
#    merged_data = merged_data.sort_values(['SID', 'valid_dttm'])#

#    # Create features and target
#    features = merged_data[['TROAD', 'glatmodel_det', 'elev', 'model_elevation', 'lat', 'lon', 'lead_time']].values
#    target = merged_data['TROAD'].values#

#    # Normalize the data
#    scaler = StandardScaler()
#    scaled_features = scaler.fit_transform(features)
#
#    return scaled_features, target, scaler



def preprocess_data(obs_data, fc_data):
    # Rename columns to avoid conflicts
    obs_data = obs_data.rename(columns={'lat': 'lat_obs', 'lon': 'lon_obs', 'elev': 'elev_obs'})
    fc_data = fc_data.rename(columns={'lat': 'lat_fc', 'lon': 'lon_fc', 'model_elevation': 'elev_fc'})
    
    # Merge observation and forecast data
    merged_data = pd.merge(obs_data, fc_data, on=['SID', 'valid_dttm'], suffixes=('_obs', '_fc'))
    
    # Sort data by station and time
    merged_data = merged_data.sort_values(['SID', 'valid_dttm'])
    
    # Create features and target
    feature_columns = ['TROAD', 'glatmodel_det', 'elev_obs', 'elev_fc', 'lat_obs', 'lon_obs', 'lead_time']
    features = merged_data[feature_columns].values
    target = merged_data['TROAD'].values
    
    # Normalize the data
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)
    
    return scaled_features, target, scaler

#older version

def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch_features, batch_targets in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_targets.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_features, batch_targets in val_loader:
                outputs = model(batch_features)
                loss = criterion(outputs, batch_targets.unsqueeze(1))
                val_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')


In [60]:
def train_model(model, train_loader, val_loader, num_epochs=50, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch_features, batch_targets in train_loader:
            batch_features, batch_targets = batch_features.to(device), batch_targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_targets.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_features, batch_targets in val_loader:
                batch_features, batch_targets = batch_features.to(device), batch_targets.to(device)
                
                outputs = model(batch_features)
                loss = criterion(outputs, batch_targets.unsqueeze(1))
                val_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

#older version
def evaluate_model(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for batch_features, batch_targets in test_loader:
            outputs = model(batch_features)
            predictions.extend(outputs.numpy().flatten())
            actuals.extend(batch_targets.numpy().flatten())

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))[:, 0]
    actuals = scaler.inverse_transform(np.array(actuals).reshape(-1, 1))[:, 0]

    mae = np.mean(np.abs(predictions - actuals))
    rmse = np.sqrt(np.mean((predictions - actuals)**2))

    return mae, rmse


In [61]:
def evaluate_model(model, test_loader, scaler):
    model.eval()
    predictions = []
    actuals = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    with torch.no_grad():
        for batch_features, batch_targets in test_loader:
            batch_features, batch_targets = batch_features.to(device), batch_targets.to(device)
            
            outputs = model(batch_features)
            predictions.extend(outputs.cpu().numpy().flatten())
            actuals.extend(batch_targets.cpu().numpy().flatten())
    
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))[:, 0]
    actuals = scaler.inverse_transform(np.array(actuals).reshape(-1, 1))[:, 0]
    
    mae = np.mean(np.abs(predictions - actuals))
    rmse = np.sqrt(np.mean((predictions - actuals)**2))
    
    return mae, rmse

In [63]:
def time_based_split(dataset, train_ratio=0.7, val_ratio=0.15):
    total_size = len(dataset)
    train_size = int(train_ratio * total_size)
    val_size = int(val_ratio * total_size)

    train_dataset = Subset(dataset, range(train_size))
    val_dataset = Subset(dataset, range(train_size, train_size + val_size))
    test_dataset = Subset(dataset, range(train_size + val_size, total_size))

    return train_dataset, val_dataset, test_dataset


In [64]:
#variables = ['TROAD', 'T2M', 'WS', 'WD', 'RH', 'PREC']  # Add all your variables here

variables = ['TROAD', 'T2m', 'Td2m', 'D10m', 'S10m', 'AccPcp12h']  # Add all your variables here

OB_PATH = "/media/cap/extra_work/road_model/OBSTABLE"
FC_PATH = "/media/cap/extra_work/road_model/FCTABLE"

year = 2023  # Change this to the year you want to analyze
month = 1  # Change this to the month you want to analyze
init_time = 0  # Change this to the initialization time you want to use

# Load observation and forecast data
obs_data = load_observation_data(OB_PATH,variables, year)
fc_data = load_forecast_data(FC_PATH,variables, year, month, init_time)



In [65]:
obs_data.tail(10)

,valid_dttm,SID,lat,lon,elev,TROAD,variable,T2m,Td2m,D10m,S10m,AccPcp12h
456182,2023-12-20 13:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,188.54
456183,2023-12-20 14:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,188.54
456184,2023-12-20 15:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,188.54
456185,2023-12-20 08:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,188.02
456186,2023-12-20 09:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,188.02
456187,2023-12-20 10:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,188.02
456188,2023-12-20 11:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,188.07
456189,2023-12-06 10:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,171.56
456190,2023-12-10 10:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,177.11
456191,2023-12-17 18:00:00,130000,55.782998,12.280158,23.308584,NaN,AccPcp12h,NaN,NaN,NaN,NaN,180.56


In [66]:
#obs_data.head(10)

In [67]:
# Preprocess data
features, target, scaler = preprocess_data(obs_data, fc_data)


ValueError: too many values to unpack (expected 3)

In [47]:
# Create datasets
sequence_length = 24  # Adjust as needed, this are the number of days
dataset = WeatherDataset(features, target, sequence_length)


In [48]:
# Split data into train, validation, and test sets
#train_size = int(0.7 * len(dataset))
#val_size = int(0.15 * len(dataset))
#test_size = len(dataset) - train_size - val_size
#train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])


In [ ]:
   # Split data into train, validation, and test sets based on time
    train_dataset, val_dataset, test_dataset = time_based_split(dataset)


In [49]:
# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [53]:
val_dataset

In [50]:
# Initialize and train the model
input_size = features.shape[1]
hidden_size = 64
num_layers = 2
output_size = 1

model = LSTM(input_size, hidden_size, num_layers, output_size)
train_model(model, train_loader, val_loader)


Epoch [1/50], Train Loss: nan, Val Loss: nan


KeyboardInterrupt: 